In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Import water quality data 
data = pd.read_csv('2017 Water Quality Archive.csv') 

# Check dataframe head 
print("Dataframe head：")
print(data.head(),"\n")

# Check dataframe shape 
print("Dataframe shape ：")
print(data.shape,"\n")

# Check data types and not Null value
print("Data types and not Null value：")
print(data.info(),"\n")

# Check for missing values 
print("Missing values：")
print(data.isnull().sum(),"\n")

#Check uniqueness of columns 
print("Number of unique values per column：")
print(data.nunique(),"\n")

# Check for duplicate rows 
print(data[data.duplicated()].shape[0], "\n")


In [ ]:

import pandas as pd
import numpy as np

# Specify the range of years to be processed
years = range(2000, 2023)  # From year 2000 to 2022

for year in years:
    input_file_name = f"{year} Water Quality Archive.csv"
    output_file_name = f"{year} Water Quality Archive Cleaned.csv"
    
    try:
        df = pd.read_csv(input_file_name)
    except FileNotFoundError:
        print(f"File {input_file_name} not found. Skipping this year.")
        continue
    except Exception as e:
        print(f"An error occurred while reading {input_file_name}: {e}")
        continue

    print(f"{input_file_name} shape before processing: {df.shape}")

    # Check and remove rows with non-NA 'deviating_result' values if the column exists
    if 'deviating_result' in df.columns:
        initial_count = df.shape[0]
        df = df[df['deviating_result'].isna()]
        final_count = df.shape[0]
        print(f"Removed {initial_count - final_count} rows with non-NA 'deviating_result'")
    else:
        print("'deviating_result' column not found in the data.")

    # Remove specific columns and handle their absence gracefully
    for col in ['deviating_result', 'sign', 'unit_name']:
        if col in df.columns:
            df = df.drop(columns=[col])
        else:
            print(f"{col} column not present in {input_file_name}.")

    # Attempt to convert 'parameter_shortname' to int and handle errors
    if 'parameter_shortname' in df.columns:
        try:
            df['parameter_shortname'] = df['parameter_shortname'].astype(int)
        except ValueError:
            print(f"Cannot convert 'parameter_shortname' to int in {input_file_name}")

    # Handling 'unit_symbol' replacement
    if 'unit_symbol' in df.columns:
        df['unit_symbol'] = df['unit_symbol'].replace('---', 'scalar')
    else:
        print("'unit_symbol' column not found in the data.")

    # Handling 'sample_value' conversions and deletions for erroneous entries
    if 'sample_value' in df.columns and df['sample_value'].dtype == object:
        original_sample_values = df['sample_value'].copy()  # Save original values for comparison
        df['sample_value'] = pd.to_numeric(df['sample_value'], errors='coerce')
        df.dropna(subset=['sample_value'], inplace=True)
        print(f"Processed 'sample_value' conversions. Data type: {df['sample_value'].dtype}")

    print(f"{input_file_name} shape after processing: {df.shape}")
    try:
        df.to_csv(output_file_name, index=False)  # Save the modified data to a new file
        print(f"{output_file_name} processed and saved successfully.")
    except Exception as e:
        print(f"Failed to save {output_file_name}: {e}")

    print(f"Data types after processing {output_file_name}:\n{df.dtypes}\n")


In [ ]:
# This code reads water quality data files from 2000 to 2023, provides an overview of each dataset,
# and collects summary statistics such as the number of rows, columns, missing values, and unique values.
# It then stores the summary information in a DataFrame for further analysis or visualization.

import pandas as pd
import matplotlib.pyplot as plt

years = range(2000, 2024)
data_summary = []

for year in years:
    
    df = pd.read_csv(f'{year} Water Quality Archive.csv')
    
    # Optionally, compare with the cleaned data
    # df = pd.read_csv(f'{year} Water Quality Archive Cleaned.csv')
    
    # Print basic information for each year's dataset
    print(f"Year: {year}")
    print("Head of data:\n", df.head())           # Display first few rows
    print("Shape of data:", df.shape)             # Show number of rows and columns
    print("Data types:\n", df.dtypes)             # Show column data types
    
    # Calculate missing values for each column
    missing_values = df.isnull().sum()
    print("Missing values:\n", missing_values)
    
    # Calculate the number of unique values for each column
    unique_values = df.nunique()
    print("Number of unique values per column:\n", unique_values)
    
    # Append summary information (year, rows, columns, missing and unique values) to the list
    data_summary.append({
        'Year': year,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Missing Values': missing_values.sum(),
        'Unique Values': unique_values.sum()
    })

# Convert the collected summary data into a DataFrame for further use or visualization
summary_df = pd.DataFrame(data_summary)



In [ ]:
# This script processes water quality data files from 2000 to 2023. 
# It checks for consistency in the column names and count across all the years, comparing them with the first year as a baseline.
# Additionally, it identifies and summarizes any missing values in the columns for each year.
# The script outputs any discrepancies in column structure and missing value information, providing a comprehensive overview of the data quality.

import pandas as pd

# Define the range of years to be processed
years = range(2000, 2024)

# Initialize variables to store column information for the first year for comparison
initial_columns = None
initial_column_count = None
column_consistency = True

# Dictionary to store missing values information across years
missing_values_summary = {}

# Process each file
for year in years:
    file_name = f'{year} Water Quality Archive Cleaned.csv'
    try:
        df = pd.read_csv(file_name)
    except FileNotFoundError:
        print(f"File {file_name} not found. Skipping this year.")
        continue

    # Retrieve current year column information
    current_columns = df.columns
    current_column_count = len(current_columns)

    # Initialize column comparison in the first year
    if initial_columns is None:
        initial_columns = current_columns
        initial_column_count = current_column_count
        print(f"Baseline set with {initial_column_count} columns from the year {year}.")
    else:
        # Compare the number of columns and the column names with the baseline year
        if current_column_count != initial_column_count or set(current_columns) != set(initial_columns):
            column_consistency = False
            print(f"Year {year}: Column discrepancies detected.")
            if current_column_count != initial_column_count:
                print(f"  - Different number of columns: {current_column_count} (expected {initial_column_count}).")
            if set(current_columns) != set(initial_columns):
                print(f"  - Different column names.")

    # Check for missing values
    missing_values = df.isna().sum()
    for column, count in missing_values.items():
        if count > 0:
            if column not in missing_values_summary:
                missing_values_summary[column] = {}
            missing_values_summary[column][year] = count

    #print()  # Add a space for better readability between years

# Check if all years had the same columns if no discrepancies were noted
if column_consistency:
    print("All years have consistent columns.")

# Print the summary of missing values
if missing_values_summary:
    print("Summary of Missing Values by Column Across All Years:")
    for column, details in missing_values_summary.items():
        print(f"Column: {column}")
        for year, count in details.items():
            print(f"  Year {year}: {count} missing values")
else:
    print("No missing values found across any year.")

In [ ]:
import pandas as pd

# Set the range of years for the analysis (from 2000 to 2023)
years = range(2000, 2024)

# Define the columns to be analyzed for unique values
columns_to_analyze = [
    'station_type', 'local_authority', 'sampling_reason', 'reason_group', 'sampling_medium', 
    'sampling_mechanism', 'parameter_shortname', 'parameter_name', 'method_name',
    'coded_value'
]

# Create a dictionary to store the unique values for each column
unique_values = {col: set() for col in columns_to_analyze}

# Loop through each year to process the respective file
for year in years:
    # Construct the file name and read the data
    file_name = f'{year} Water Quality Archive Cleaned.csv'
    try:
        df = pd.read_csv(file_name)
        
        # Update the unique values set for each column
        for col in columns_to_analyze:
            if col in df.columns:
                unique_values[col].update(df[col].dropna().unique())
            else:
                # Print a message if the column is not found in the current file
                print(f"Column {col} not found in {file_name}")
    except FileNotFoundError:
        # Handle the case where the file does not exist
        print(f"File {file_name} not found. Skipping this year.")

# Prepare the results for saving as a CSV file
output_data = {}
for col, values in unique_values.items():
    output_data[col] = list(values)

# Convert the dictionary of unique values to a DataFrame
output_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in output_data.items()]))

# Save the DataFrame to a CSV file
output_df.to_csv('unique_values_summary.csv', index=False)

# Print the number of unique values and the unique values for each column
for col, values in unique_values.items():
    print("\n")  # Ensure space between each column's output
    print(f"Column '{col}' has {len(values)} unique values.")
    print(f"Unique values for '{col}': {values}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re  


years = range(2000, 2024)
# Dictionary to map specific station types to broader categories, ensuring all text is lowercase
category_map = {


    'Waste Sites': [
        'waste site leachate borehole', 'waste site gas groundwater', 'waste site domestic industrial landfill',
        'waste site unspecified', 'waste site external standing water inc ponds ditches', 'waste site treated effluent',
        'waste site gas borehole', 'waste site leachate management system', 'waste site surface water monitoring point landfill'
    ],
    'Trade Discharges': [
        'trade discharges process effluent not water company', 'trade discharges mineral workings',
        'trade discharges process effluent water company wtw', 'trade discharges site drainage contam surface water not waste sit',
        'trade discharges site drainage', 'trade discharges abandoned', 'trade discharges cooling water', 'trade discharges unspecified',
        'trade release to air'
    ],
    'Freshwater Sites': [
        'freshwater canals non classified', 'freshwater rqo re1', 'freshwater comparative inlet points', 'freshwater land drains',
        'freshwater lakes ponds reservoirs', 'freshwater non classified river points', 'freshwater unspecified', 'freshwater rqo re2',
        'freshwater rqo re3', 'freshwater bathing water'
    ],
    'Sewage Discharges': [
        'sewage discharges unspecified water company', 'sewage crude to further treatment water company', 
        'sewage discharges stw storm overflow storm tank not water company', 'sewage discharges sludge water company',
        'sewage discharges stw storm overflow storm tank water company', 'sewage discharges final treated effluent not water company',
        'sewage discharges final treated effluent water company', 'sewage discharges pumping station water company', 
        'sewage trade combined unspecified', 'sewage discharges sewer storm overflow water company', 
        'sewage discharges unspecified not water company', 'sewage discharges sewer storm overflow not water company',
        'sewerage system discharge'
    ],
    'Saline Water Sites': [
        'saline water designated shellfisheries', 'saline water estuarine sites corrected',
        'saline water estuary class c', 'saline water non designated bathing beaches', 'saline water estuarine sites non bathing shellfish',
        'saline water estuary class a', 'saline water comparative inlet point', 'saline water designated bathing beaches',
        'saline water coastal sites non bathing shellfish', 'saline water estuary class b'
    ],
    'Agriculture Sites': [
        'agriculture unspecified', 'agriculture site drainage', 'agriculture fish farming not water company',
        'agriculture fish farming water company'
    ],
    'Miscellaneous Sites': [
        'miscellaneous environment sediments', 'miscellaneous discharges surface water',
        'miscellaneous discharges highway drainage', 'miscellaneous environment unspecified', 
        'miscellaneous discharges unspecified', 'miscellaneous environment soils', 'miscellaneous discharges mine groundwater as raised'
    ],
    'Other Categories': [
        'pollution investigation points environment', 'rainwater', 'unspecified'
    ],
    'Minewater': [
        'minewater'
    ]
}

# Function
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text.lower()

# Function to map station types to broader categories
def map_category(station_type):
    cleaned_type = clean_text(station_type)
    for category, types in category_map.items():
        if cleaned_type in types:
            return category
            
    print(f"Warning: '{station_type}' did not match any category.")  
    return 'Other Categories'

  
# List to store data frames for each year
data_frames = []

# Loop through each year and read the data files
for year in years:
    file_name = f'{year} Water Quality Archive Cleaned.csv'
    try:
        df = pd.read_csv(file_name)
        if 'station_number' in df.columns and 'station_type' in df.columns:
            # Clean station_type text and map to category
            df['station_type'] = df['station_type'].astype(str).apply(clean_text)
            df['category'] = df['station_type'].apply(map_category)
            df = df[['station_number', 'category']]  # Keep relevant columns
            df['year'] = year  # Add year column
            data_frames.append(df)  # Append to the list
        else:
            print(f"Required columns are missing in {file_name}")
    except FileNotFoundError:
        print(f"File {file_name} not found. Skipping this year.")

# Combine all yearly data frames into a single data frame
full_data = pd.concat(data_frames)

# Count the number of stations per category for each year
category_counts = full_data.groupby(['year', 'category']).size().unstack(fill_value=0)

# Save the result as a CSV file
category_counts.to_csv('station_type_category_counts_by_year.csv')

# Plotting the change in station counts over the years
plt.figure(figsize=(20, 10))
for category in category_counts.columns:
    plt.plot(category_counts.index, category_counts[category], marker='o', label=category)

# Add labels, title, and legend to the plot
plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Stations by Category (2000-2023)')
plt.legend(title='Station Type Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Display the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the aggregated category counts from the CSV file
category_counts = pd.read_csv('station_type_category_counts_by_year.csv', index_col='year')

# Separate data into different category groups for plotting
freshwater_counts = category_counts['Freshwater Sites']
saline_sewage_counts = category_counts[['Saline Water Sites', 'Sewage Discharges']]
waste_counts = category_counts['Waste Sites']
agriculture_other_counts = category_counts[['Agriculture Sites', 'Other Categories']]
other_counts = category_counts.drop(columns=['Freshwater Sites', 'Saline Water Sites', 'Sewage Discharges', 
                                            'Waste Sites', 'Agriculture Sites', 'Other Categories'])

# Plot the number of Freshwater Sites over the years
plt.figure(figsize=(20, 6))
plt.plot(freshwater_counts.index, freshwater_counts, marker='o', label='Freshwater Sites', color='blue')
plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Freshwater Sites (2000-2023)')
plt.legend(loc='upper left')
plt.show()

# Plot the number of Saline Water Sites and Sewage Discharges over the years
plt.figure(figsize=(20, 6))
for category in saline_sewage_counts.columns:
    plt.plot(saline_sewage_counts.index, saline_sewage_counts[category], marker='o', label=category)

plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Saline Water Sites and Sewage Discharges (2000-2023)')
plt.legend(title='Station Type Category', loc='upper left')
plt.show()

# Plot the number of Waste Sites over the years
plt.figure(figsize=(20, 6))
plt.plot(waste_counts.index, waste_counts, marker='o', label='Waste Sites', color='green')
plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Waste Sites (2000-2023)')
plt.legend(loc='upper left')
plt.show()

# Plot the number of Agriculture Sites and Other Categories over the years
plt.figure(figsize=(20, 6))
for category in agriculture_other_counts.columns:
    plt.plot(agriculture_other_counts.index, agriculture_other_counts[category], marker='o', label=category)

plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Agriculture Sites and Other Categories (2000-2023)')
plt.legend(title='Station Type Category', loc='upper left')
plt.show()

# Plot the number of stations in other categories over the years
plt.figure(figsize=(20, 10))
for category in other_counts.columns:
    plt.plot(other_counts.index, other_counts[category], marker='o', label=category)

plt.xlabel('Year')
plt.ylabel('Number of Stations')
plt.title('Change in Number of Stations by Other Categories (2000-2023)')
plt.legend(title='Station Type Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()



In [ ]:
import pandas as pd
import warnings

# Ignore all warnings to prevent unnecessary output
warnings.filterwarnings("ignore")

# Initialize dictionaries to store station information and stations with one-to-many mappings
total_station_info = {}
total_one_to_many = {}

# Define the range of years for which data will be processed
years = range(2000, 2024)

# Loop through each year and process the data
for year in years:
    df = pd.read_csv(f'{year} Water Quality Archive Cleaned.csv')
    
    # Group the data by station number and check for duplicates in specific columns
    for station_number, group in df.groupby('station_number'):
        if len(group[['station_name', 'station_type', 'northing', 'easting', 'ngr']].drop_duplicates()) == 1:
            # If no duplicates, store or update the station information
            row = group.iloc[0]
            total_station_info[station_number] = {
                'Station Name': row['station_name'],
                'Station Type': row['station_type'],
                'Northing': row['northing'],
                'Easting': row['easting'],
                'ngr': row['ngr']
            }
        else:
            # If duplicates exist (one-to-many mapping), add the station number to the one-to-many dictionary
            if station_number not in total_one_to_many:
                total_one_to_many[station_number] = []
            total_one_to_many[station_number].append(year)

# Save the station information as a CSV file, overwriting the existing file if it exists
station_info_df = pd.DataFrame.from_dict(total_station_info, orient='index')
station_info_df.to_csv('total_station_info.csv', mode='w')

# Save the one-to-many mapping as a separate CSV file
one_to_many_df = pd.DataFrame([(station, years) for station, years in total_one_to_many.items()],
                              columns=['Station Number', 'Years'])
one_to_many_df.to_csv('total_one_to_many.csv', mode='w', index=False)

# Print the number of entries in both data sets for confirmation
station_info_count = len(station_info_df)
print(f"Total entries in station_info: {station_info_count}")

one_to_many_count = len(one_to_many_df)
print(f"\nTotal entries in one_to_many: {one_to_many_count}")



In [ ]:
from IPython.display import display, HTML
import pandas as pd

# Read data from the 'total_station_info.csv' file
station_info_df = pd.read_csv('total_station_info.csv', index_col=0)

# Sort the DataFrame by 'Station Name' in ascending order (A-Z)
sorted_station_info_df = station_info_df.sort_values(by='Station Name')

# Print the total number of entries in the sorted DataFrame
print(f"Total entries in station_info: {len(sorted_station_info_df)}")

# Display the sorted DataFrame as an HTML table with the index column visible
print("Sorted Station Info Table by Station Name (A-Z):")
display(HTML(sorted_station_info_df.to_html(index=True)))



In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Read the saved CSV file containing station information
station_info_df = pd.read_csv('total_station_info.csv', index_col=0)

# Group the data by 'Northing' and 'Easting' to calculate the number of stations at each location
location_groups = station_info_df.groupby(['Northing', 'Easting']).size()

# Filter the groups where the location has more than one station (duplicates)
duplicates = location_groups[location_groups > 1]

# Display the results
print("Duplicate Locations (Northing and Easting with more than one station):")
if not duplicates.empty:
    for location, count in duplicates.items():
        print(f"Location (Northing: {location[0]}, Easting: {location[1]}) - Stations: {count}")
        # Display the station details for these duplicate locations in HTML format
        display(HTML(station_info_df[(station_info_df['Northing'] == location[0]) & 
                                     (station_info_df['Easting'] == location[1])].to_html(index=False)))
else:
    print("No duplicate locations found.")



In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Load the CSV file containing station information
station_info_df = pd.read_csv('total_station_info.csv', index_col=0)

# Group the data by 'Northing' and 'Easting' to count how many stations are located at each unique position
location_groups = station_info_df.groupby(['Northing', 'Easting']).size()

# Filter out locations where more than one station exists (duplicates)
duplicates = location_groups[location_groups > 1]

# Initialize an empty DataFrame to store details about stations at duplicate locations
all_duplicate_info_df = pd.DataFrame()

# Display and collect duplicate station details
if not duplicates.empty:
    print("Duplicate Locations (Northing and Easting with more than one station):")
    for location, count in duplicates.items():
        print(f"Location (Northing: {location[0]}, Easting: {location[1]}) - Stations: {count}")
        # Extract the stations for the given duplicate location
        stations_at_location = station_info_df[(station_info_df['Northing'] == location[0]) & 
                                               (station_info_df['Easting'] == location[1])]
        # Display the station details in an HTML table
        display(HTML(stations_at_location.to_html(index=False)))
        # Append the duplicate station details to the DataFrame
        all_duplicate_info_df = pd.concat([all_duplicate_info_df, stations_at_location], ignore_index=True)
else:
    print("No duplicate locations found.")

# Save the collected duplicate station information to a new CSV file
all_duplicate_info_df.to_csv('all_duplicate_stations_info.csv', index=False)

print("All duplicate station information has been saved to 'all_duplicate_stations_info.csv'.")


In [ ]:
import pandas as pd

# Load the CSV file containing station information
station_info_df = pd.read_csv('total_station_info.csv', index_col=0)

# Standardize the 'Station Type' column by stripping leading/trailing spaces and converting text to lowercase
station_info_df['Station Type'] = station_info_df['Station Type'].str.strip().str.lower()

# Filter the DataFrame to select only rows where 'Station Type' contains 'minewater'
minewater_stations_df = station_info_df[station_info_df['Station Type'].str.contains('minewater', na=False)]

# Save the filtered minewater stations information to a new CSV file
minewater_stations_df.to_csv('minewater_stations_info.csv', index=False)

# Display the filtered minewater stations information
print("Minewater Stations Information:")
print(minewater_stations_df)


In [ ]:
# Checking for any mismatches between these two columns
parameter_check = data[['parameter_shortname', 'parameter_name']].drop_duplicates()
print("Unique Parameter Check:", parameter_check.shape[0])


In [ ]:
import pandas as pd
import re

# Define a text cleaning function to remove special characters and normalize spaces
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text.lower()

# Set the range of years to process
years = range(2000, 2023 + 1)

# Initialize Series to store overall counts for 'sampling_reason' and 'reason_group'
overall_sampling_reason_counts = pd.Series(dtype=int)
overall_reason_group_counts = pd.Series(dtype=int)

# Loop through each year's file and process it
for year in years:
    file_name = f'{year} Water Quality Archive Cleaned.csv'
    
    try:
        df = pd.read_csv(file_name)
        
        # Check if the necessary columns exist
        if 'sampling_reason' in df.columns and 'reason_group' in df.columns:
            # Clean the text data in the relevant columns
            df['sampling_reason'] = df['sampling_reason'].apply(clean_text)
            df['reason_group'] = df['reason_group'].apply(clean_text)

            # Count the occurrences of each 'sampling_reason' and 'reason_group'
            sampling_reason_counts = df['sampling_reason'].value_counts()
            reason_group_counts = df['reason_group'].value_counts()

            # Add the counts to the overall totals
            overall_sampling_reason_counts = overall_sampling_reason_counts.add(sampling_reason_counts, fill_value=0)
            overall_reason_group_counts = overall_reason_group_counts.add(reason_group_counts, fill_value=0)
        
        else:
            print(f"File {file_name} missing required columns, skipping...")
    
    except FileNotFoundError:
        print(f"File {file_name} not found, skipping...")

# Save the total counts to CSV files
overall_sampling_reason_counts.to_csv('total_sampling_reason_counts_2000_2023.csv')
overall_reason_group_counts.to_csv('total_reason_group_counts_2000_2023.csv')

print("Yearly data has been processed, summed, and saved.")




In [ ]:
import pandas as pd
import re
import os
import glob
from scipy.stats import chi2_contingency
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# This script processes water quality data from multiple CSV files, 
# cleans the text data, performs statistical analysis, and saves the results.

# Define a function to clean text data
def clean_text(text):
    try:
        cleaned_text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        return cleaned_text.lower()
    except Exception as e:
        print(f"Error cleaning text: {e}")
        return text

# Set the range of years to process
years = range(2000, 2023 + 1)

# Initialize Series to store overall counts
overall_sampling_reason_counts = pd.Series(dtype='int32')
overall_reason_group_counts = pd.Series(dtype='int32')

# Initialize a DataFrame to accumulate all files' data
combined_df = pd.DataFrame()

# Read and process each file for the specified years
for year in years:
    file_name = f'{year} Water Quality Archive Cleaned.csv'
    
    try:
        # Load the data with specified data types to optimize memory usage
        df = pd.read_csv(file_name, dtype={'sampling_reason': 'category', 'reason_group': 'category'})
        
        if 'sampling_reason' in df.columns and 'reason_group' in df.columns:
            try:
                # Clean the text columns
                df['sampling_reason'] = df['sampling_reason'].apply(clean_text)
                df['reason_group'] = df['reason_group'].apply(clean_text)

                # Calculate counts for the current year
                sampling_reason_counts = df['sampling_reason'].value_counts().astype('int32')
                reason_group_counts = df['reason_group'].value_counts().astype('int32')

                # Accumulate the counts into overall statistics
                overall_sampling_reason_counts = overall_sampling_reason_counts.add(sampling_reason_counts, fill_value=0).astype('int32')
                overall_reason_group_counts = overall_reason_group_counts.add(reason_group_counts, fill_value=0).astype('int32')

                # Add the current year's data to the combined DataFrame
                combined_df = pd.concat([combined_df, df[['sampling_reason', 'reason_group']]], ignore_index=True)

                # Release memory
                del df, sampling_reason_counts, reason_group_counts

                # Print progress
                print(f"Processed data for year {year}.")
            except Exception as e:
                print(f"Error processing data for year {year}: {e}")
        else:
            print(f"File {file_name} missing required columns, skipping...")
    
    except FileNotFoundError:
        print(f"File {file_name} not found, skipping...")
    except pd.errors.EmptyDataError:
        print(f"File {file_name} is empty, skipping...")
    except Exception as e:
        print(f"Error loading file {file_name}: {e}")

# Save overall counts as CSV files
try:
    overall_sampling_reason_counts.to_csv('total_sampling_reason_counts_2000_2023.csv')
    print("Saved total_sampling_reason_counts_2000_2023.csv successfully.")

    overall_reason_group_counts.to_csv('total_reason_group_counts_2000_2023.csv')
    print("Saved total_reason_group_counts_2000_2023.csv successfully.")
except Exception as e:
    print(f"Error saving total counts: {e}")

# Create and save a cross-tabulation of the data
try:
    cross_tab = pd.crosstab(combined_df['sampling_reason'], combined_df['reason_group'])

    cross_tab.to_csv('cross_tab_sampling_reason_vs_reason_group.csv')
    print("Saved cross_tab_sampling_reason_vs_reason_group.csv successfully.")
except Exception as e:
    print(f"Error creating or saving cross tabulation: {e}")

# Perform chi-squared test and save the results
try:
    chi2, p, dof, expected = chi2_contingency(cross_tab)

    with open('chi_squared_test_results.txt', 'w') as f:
        f.write(f"Chi2 Statistic: {chi2:.2f}\n")
        f.write(f"P-value: {p:.4f}\n")
        f.write(f"Degrees of Freedom: {dof}\n")
        f.write("Expected Frequencies Table:\n")
        f.write(pd.DataFrame(expected, index=cross_tab.index, columns=cross_tab.columns).to_string())
    print("Saved chi_squared_test_results.txt successfully.")

    pd.DataFrame(expected, index=cross_tab.index, columns=cross_tab.columns).to_csv('expected_frequencies.csv')
    print("Saved expected_frequencies.csv successfully.")
except Exception as e:
    print(f"Error during chi-squared test or saving results: {e}")

# Print final results
print("Yearly data has been processed, summed, and saved.")
print("\nChi-squared Test Results:")
print(f"Chi2 Statistic: {chi2:.2f}")
print(f"P-value: {p:.4f}")
print(f"Degrees of Freedom: {dof}")

# Interpretation of chi-squared test results
if p < 0.05:
    print("The relationship between sampling_reason and reason_group is statistically significant.")
else:
    print("No significant relationship between sampling_reason and reason_group.")




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# This script loads a contingency table from a CSV file, 
# visualizes it as a heatmap, and then computes and visualizes 
# the residuals (observed - expected frequencies) from a chi-squared test.

# 1. Load the contingency table from a CSV file
cross_tab = pd.read_csv('cross_tab_sampling_reason_vs_reason_group.csv', index_col=0)

# 2. Visualization: Heatmap of the contingency table
plt.figure(figsize=(12, 8))
sns.heatmap(cross_tab, annot=True, cmap='YlGnBu', fmt='d', cbar=True)
plt.title('Heatmap of Sampling Reason vs Reason Group')
plt.xlabel('Reason Group')
plt.ylabel('Sampling Reason')
plt.show()

# 3. Load expected frequencies from the generated CSV file
expected = pd.read_csv('expected_frequencies.csv', index_col=0)

# 4. Calculate the residuals (observed - expected values)
residuals = cross_tab - expected

# 5. Visualization: Heatmap of the residuals
plt.figure(figsize=(12, 8))
sns.heatmap(residuals, annot=True, cmap='RdBu_r', center=0, cbar=True)
plt.title('Residuals Heatmap: Observed - Expected Frequencies')
plt.xlabel('Reason Group')
plt.ylabel('Sampling Reason')
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This script reads total sampling reason counts, separates them based on frequency, 
# and visualizes high and low frequency counts using bar plots.

# Read overall sampling reason counts from the CSV file
overall_sampling_reason_counts = pd.read_csv('total_sampling_reason_counts_2000_2023.csv', index_col=0, squeeze=True)

# Separate counts into high and low frequency based on a threshold of 300,000
high_sampling_reason_counts = overall_sampling_reason_counts[overall_sampling_reason_counts >= 300000]
low_sampling_reason_counts = overall_sampling_reason_counts[overall_sampling_reason_counts < 300000]

# Plot bar chart for high frequency sampling reasons (>= 300,000)
plt.figure(figsize=(14, 7))
high_sampling_reason_counts.plot(kind='bar', color=sns.color_palette('Paired', len(high_sampling_reason_counts)))
plt.title('Sampling Reason Counts (2000-2023) - High Frequency (>= 300,000)')
plt.xlabel('Sampling Reason')
plt.ylabel('Total Count')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Plot bar chart for low frequency sampling reasons (< 300,000)
plt.figure(figsize=(14, 7))
low_sampling_reason_counts.plot(kind='bar', color=sns.color_palette('Set3', len(low_sampling_reason_counts)))
plt.title('Sampling Reason Counts (2000-2023) - Low Frequency (< 300,000)')
plt.xlabel('Sampling Reason')
plt.ylabel('Total Count')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This script reads total reason group counts, prints the distribution for analysis, 
# and visualizes high and low frequency counts using bar plots.

# Read overall reason group counts from the CSV file
overall_reason_group_counts = pd.read_csv('total_reason_group_counts_2000_2023.csv', index_col=0, squeeze=True)

# Print the sorted distribution of reason group counts to observe and decide on thresholds
print(overall_reason_group_counts.sort_values(ascending=False))

# Divide the counts into high and low frequency based on a threshold of 1,000,000
high_reason_group_counts = overall_reason_group_counts[overall_reason_group_counts >= 1000000]
low_reason_group_counts = overall_reason_group_counts[overall_reason_group_counts < 1000000]

# Plot bar chart for high frequency reason groups (>= 1,000,000)
plt.figure(figsize=(10, 6))
high_reason_group_counts.plot(kind='bar', color=sns.color_palette('Set2', len(high_reason_group_counts)))
plt.title('Reason Group Counts (2000-2023) - High Frequency (>= 1,000,000)')
plt.xlabel('Reason Group')
plt.ylabel('Total Count')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Plot bar chart for low frequency reason groups (< 1,000,000)
plt.figure(figsize=(10, 6))
low_reason_group_counts.plot(kind='bar', color=sns.color_palette('Set3', len(low_reason_group_counts)))
plt.title('Reason Group Counts (2000-2023) - Low Frequency (< 1,000,000)')
plt.xlabel('Reason Group')
plt.ylabel('Total Count')
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd

# This script analyzes water quality data across years, looking for inconsistencies 
# in parameter names and their corresponding short names and unit symbols.

# Set the range of years for analysis
years = range(2000, 2024)  # From 2000 to 2023

# List to store inconsistencies from all years
all_inconsistencies = []

for year in years:
    input_file_name = f"{year} Water Quality Archive Cleaned.csv"
    
    # Try to read the CSV file for the current year
    try:
        df = pd.read_csv(input_file_name)
    except FileNotFoundError:
        print(f"File {input_file_name} not found. Skipping this year.")
        continue
    except Exception as e:
        print(f"An error occurred while reading {input_file_name}: {e}")
        continue

    # Group the data by 'parameter_name' to check for inconsistencies
    grouped = df.groupby('parameter_name')
    inconsistencies = []

    for name, group in grouped:
        unique_shortnames = group['parameter_shortname'].unique()
        if len(unique_shortnames) > 1:
            # Collect unique unit symbols for each short name
            unit_symbols = group.groupby('parameter_shortname')['unit_symbol'].unique().to_dict()
            inconsistencies.append({
                'parameter_name': name,
                'parameter_shortnames': unique_shortnames,
                'unit_symbols': unit_symbols
            })

    # Record inconsistencies if found
    if inconsistencies:
        print(f"Inconsistencies found for year {year}:")
        for inc in inconsistencies:
            print(f"Parameter Name: {inc['parameter_name']}")
            for shortname, units in inc['unit_symbols'].items():
                print(f"  Shortname: {shortname}, Units: {units}")
        all_inconsistencies.extend(inconsistencies)
    else:
        print(f"No inconsistencies found for year {year}.")

# Output all inconsistencies to a CSV file
if all_inconsistencies:
    # Create a DataFrame from the list of inconsistencies
    df_inconsistencies = pd.DataFrame(all_inconsistencies)
    # Save the DataFrame to a CSV file
    df_inconsistencies.to_csv("Inconsistencies Summary.csv", index=False)
    print("Inconsistencies summary saved to 'Inconsistencies Summary.csv'.")
else:
    print("No inconsistencies found across all years.")




In [ ]:
import pandas as pd

# Define a function to handle unit conversions
def convert_units(value, from_unit, to_unit, parameter_name=None):
    conversion_factors = {
        ('mg/kg', 'µg/kg'): 1000,
        ('µg/kg', 'mg/kg'): 0.001,
        ('mg/l', 'µg/l'): 1000,
        ('µg/l', 'mg/l'): 0.001,
        ('g/kg', 'mg/l'): 1000,
        ('µS/cm', 'mS/cm'): 0.001,
        ('mS/cm', 'µS/cm'): 1000,
        ('ng/l', 'µg/l'): 0.001,
        ('µg/l', 'ng/l'): 1000,
        ('NTU', 'FTU'): 1,
        ('ppt', 'g/kg'): 1,
        ('Ml/d', 'm³/s'): 0.011574,
        ('m³/s', 'Ml/d'): 1 / 0.011574,
        ('µg/l', 'pg/l'): 1_000_000,  # New conversion factor
        ('pg/l', 'µg/l'): 1 / 1_000_000  # Reverse conversion
    }
    
    if parameter_name:
        # Special case for converting µg/l to µmol/l based on molecular weight
        if from_unit == 'µg/l' and to_unit == 'µmol/l':
            molecular_weights = {
                'Phosphate as P': 31.0,  # Molecular weight for phosphate
                # More molecular weights can be added as needed
            }
            if parameter_name in molecular_weights:
                return value / molecular_weights[parameter_name]
            else:
                print(f"Warning: No molecular weight provided for {parameter_name}")
                return None
        
        # Handling µg to µg/l conversion, assuming 1L volume
        if from_unit == 'µg' and to_unit == 'µg/l':
            return value  # Assuming volume of 1L
    
    # Standard conversion if rule exists
    if (from_unit, to_unit) in conversion_factors:
        return value * conversion_factors[(from_unit, to_unit)]
    elif (to_unit, from_unit) in conversion_factors:
        return value / conversion_factors[(to_unit, from_unit)]
    else:
        print(f"Warning: No conversion rule for {from_unit} to {to_unit}")
        return None

# Define the range of years to analyze
years = range(2000, 2024)
all_unit_differences = []

# Step 1: Find parameters with different units in the same year and count the unit occurrences
for year in years:
    input_file_name = f"{year} Water Quality Archive Cleaned.csv"
    
    try:
        df = pd.read_csv(input_file_name)
    except FileNotFoundError:
        print(f"File {input_file_name} not found. Skipping this year.")
        continue
    except Exception as e:
        print(f"An error occurred while reading {input_file_name}: {e}")
        continue

    grouped = df.groupby('parameter_name')
    unit_differences = []

    for name, group in grouped:
        unit_counts = group['unit_symbol'].value_counts()
        if len(unit_counts) > 1:  # Check if multiple units exist for a parameter
            unit_differences.append({
                'parameter_name': name,
                'unit_counts': unit_counts.to_dict()
            })

    if unit_differences:
        all_unit_differences.extend(unit_differences)

# Step 2 and 3: Convert minority unit values to the majority unit and update the dataset
for year in years:
    input_file_name = f"{year} Water Quality Archive Cleaned.csv"
    output_file_name = f"{year} Water Quality Archive Unified.csv"
    
    try:
        df = pd.read_csv(input_file_name)
    except FileNotFoundError:
        print(f"File {input_file_name} not found. Skipping this year.")
        continue
    except Exception as e:
        print(f"An error occurred while reading {input_file_name}: {e}")
        continue

    for ud in all_unit_differences:
        parameter_name = ud['parameter_name']
        unit_counts = ud['unit_counts']
        
        majority_unit = max(unit_counts, key=unit_counts.get)  # Find the most common unit
        
        for unit in unit_counts:
            if unit != majority_unit:
                mask = (df['parameter_name'] == parameter_name) & (df['unit_symbol'] == unit)
                # Special case for Cypermethrin to convert µg to µg/l
                if parameter_name == 'Cypermethrin' and unit == 'µg':
                    df.loc[mask, 'unit_symbol'] = 'µg/l'
                else:
                    df.loc[mask, 'sample_value'] = df.loc[mask, 'sample_value'].apply(convert_units, args=(unit, majority_unit, parameter_name))
                    df.loc[mask, 'unit_symbol'] = majority_unit

    df.to_csv(output_file_name, index=False)

# Step 4: Verify that all units are now consistent for each parameter
for year in years:
    output_file_name = f"{year} Water Quality Archive Unified.csv"
    
    try:
        df = pd.read_csv(output_file_name)
    except FileNotFoundError:
        print(f"File {output_file_name} not found. Skipping this year.")
        continue
    except Exception as e:
        print(f"An error occurred while reading {output_file_name}: {e}")
        continue

    grouped = df.groupby('parameter_name')['unit_symbol'].nunique()
    inconsistencies = grouped[grouped > 1]  # Check for parameters with multiple units

    if not inconsistencies.empty:
        print(f"Verification failed for year {year}. Multiple units still exist for some parameters:")
        print(inconsistencies)
    else:
        print(f"Verification successful for year {year}. All units are unified.")



In [ ]:
import pandas as pd

# Initialize a dictionary to store MINEWATER station info for each year
minewater_stations_by_year = {}

# Loop through each year from 2000 to 2023
for year in range(2000, 2023 + 1):
    # Load data for the year
    file_name = f'{year} Water Quality Archive Unified.csv'
    data = pd.read_csv(file_name)
    
    # Filter for MINEWATER stations
    minewater_data = data[data['station_type'].str.contains("MINEWATER", case=False, na=False)]
    
    # Extract station number and geographical information (easting, northing)
    minewater_stations = minewater_data[['station_number', 'easting', 'northing']].drop_duplicates()
    
    # Store the year's MINEWATER stations in the dictionary
    minewater_stations_by_year[year] = minewater_stations

# Initialize a dictionary to store comparison results
comparison_results = {}

# Compare MINEWATER station positions year by year
for year in range(2000, 2022):  # Stop at 2022 to avoid index error
    current_year_stations = minewater_stations_by_year[year]
    next_year_stations = minewater_stations_by_year[year + 1]
    
    # Merge two years' data and label stations as added, removed, or unchanged
    merged_stations = pd.merge(current_year_stations, next_year_stations, on=['station_number', 'easting', 'northing'], 
                               how='outer', indicator=True)
    
    # Count added, removed, and unchanged stations
    added_stations = merged_stations[merged_stations['_merge'] == 'right_only'].shape[0]
    removed_stations = merged_stations[merged_stations['_merge'] == 'left_only'].shape[0]
    unchanged_stations = merged_stations[merged_stations['_merge'] == 'both'].shape[0]
    
    # Store the result in the dictionary
    comparison_results[year] = {
        'added': added_stations,
        'removed': removed_stations,
        'unchanged': unchanged_stations
    }

# Print or save the comparison results
for year, result in comparison_results.items():
    print(f"From {year} to {year + 1}: Added = {result['added']}, Removed = {result['removed']}, Unchanged = {result['unchanged']}")
